In [1]:
%matplotlib inline

import osmnx as ox
import matplotlib.pyplot as plt
import numpy as np
import geopy.distance
import imageio
from timeit import default_timer as timer
import pandas as pd
import seaborn as sns
import scipy
from scipy.stats import norm
import requests
import json
import os
from os.path import join, dirname, abspath
from glob import glob
import io
import pathlib
from pymongo import MongoClient
from datetime import datetime
from bson import ObjectId
from pyqtree import Index
from shapely import geometry
import random
import shapely.geometry as ge

import itertools
import networkx as nx

import shapely
import random

from shapely.geometry import LineString, Point



In [2]:
from sshtunnel import SSHTunnelForwarder
import pymongo

MONGO_HOST = "viziblezone-prod.moveodevelop.com"
REMOTE_ADDRESS = ('docdb-2019-06-13-11-43-18.cluster-cybs9fpwjg54.eu-west-1.docdb.amazonaws.com',27017)
MONGO_DB = "VizibleZone"
MONGO_USER = "ubuntu"

server = SSHTunnelForwarder(
    MONGO_HOST,
    ssh_pkey = 'C:/Users/tlang/Documents/VizibleZone/pem/viziblezone-prod.pem',
    ssh_username=MONGO_USER,
    remote_bind_address=REMOTE_ADDRESS
)

server.start()

client = MongoClient('127.0.0.1', 
                     server.local_bind_port,
                     username='viziblezone',
                     password='vz123456',
                     ssl=True,
                     ssl_match_hostname=False,
                     ssl_ca_certs='C:/Users/tlang/Documents/VizibleZone/pem/rds-combined-ca-bundle.pem',
                     authMechanism='SCRAM-SHA-1') # server.local_bind_port is assigned local port
db = client[MONGO_DB]
print(db.collection_names())



['eliqbatterytest', 'eliqsensortest', 'logs', 'sensors', 'settings', 'useractivities', 'users']


In [3]:

def convert_str_to_datetime(row):
#    print(row)
    try:
        return datetime.strptime(row['timestamps_value'], '%Y-%m-%dT%H:%M:%S.%f')
    except:
        t=row['timestamps_value']
        return datetime.strptime(t[:-3]+t[-2:], '%Y-%m-%dT%H:%M:%S.%f%z')

In [4]:

import time

def convert_str_to_datetime(row):
    t=row['timestamp_local']
    return datetime.strptime(t[:-3]+t[-2:], '%Y-%m-%dT%H:%M:%S.%f%z')




def get_id_list_from_user(user_id,  start_date, end_date, device_type=0, params={"max_speed": 0.5, "min_speed": 6, 
                                                                "min_accuracy": 15, "delta_lat": 0.0001,
                                                               "max_dist_km2min": 0.2, "gap_time": 10*60}):
    # Query mongo DB for "walking" sessions
    # returns a list of IDs
    # Version 1.02, 31-07-19
    #
    # future changes: filter based on the step counter 
    #
    # params:
    # max_speed: 0.5 m/s, the lower threshold of max speed (less than that is no movement)
    # min_speed: 6 m/s, the upper threshold of min speed (more than that is driving)
    # min_accuracy: 15 m, the lower threshold of min_accuracy (more than that is bad GPS reception)
    # delta_lat: 0.0001, minimum latitute change (about 10 m North/South)
    # max_dist_km2min: 0.2 km, maximum allowed distance to be travelled in 1 minute
    # gap_time: 10*60 sec, gap between sessions to be considered a new session
    
    try: #check date string format
        start_date=datetime.strptime(start_date, "%Y-%m-%d %H:%M:%S%z")
        end_date=datetime.strptime(end_date, "%Y-%m-%d %H:%M:%S%z")
    except:
        raise ValueError("Time string should be of format: 2019-07-28 00:00:00+0300")
        
    if device_type==0:
        agg_code= [
            {"$match": {"user_id": user_id, "createdAt": {"$gt": start_date}}}, 
            { "$project": { "speed" : "$gps.speed", 
                           "_id" : "$_id", 
                           "timestamp_local" : "$timestamp_local",
                           "gps_accuracy": "$gps.accuracy",
                           "latitude": "$gps.latitude",
                           "longitude": "$gps.longitude",
                           "elapsed_time": {"$multiply": ["$gps.number_of_samples", "$sample_period"]}
                           } },
            { "$project": { "_id": "$_id", 
                           "max_speed": { "$max": "$speed" }, 
                           "min_speed": { "$min": "$speed" }, 
                           "delta_lat": {"$subtract":[{ "$max": "$latitude" }, { "$min": "$latitude" }]},
                           "delta_long": {"$subtract":[{ "$max": "$longitude" }, { "$min": "$longitude" }]},
                           "max_lat": { "$max": "$latitude" },
                           "min_accuracy": {"$min": "$gps_accuracy"},
                           "timestamp_local" : "$timestamp_local",
                           "elapsed_time": "$elapsed_time"} },
            {"$match": {"max_speed": {"$gt": params["max_speed"]}, "min_speed": {"$lt": params["min_speed"]}, 
                        "min_accuracy": {"$lt": params["min_accuracy"]}, "delta_lat": {"$gt": params["delta_lat"]}} },
            {"$project": { "_id": "$_id", "timestamp_local" : "$timestamp_local", 
                          "elapsed_time": "$elapsed_time", "max_speed": "$max_speed", 
                          "delta_lat": "$delta_lat", "delta_long":"$delta_long", "max_lat": "$max_lat"} }                  
        ]
    else:
        agg_code=  [
            {"$match": {"user_id": user_id, "device_type": device_type, "createdAt": {"$gt": start_date}}}, 
            { "$project": { "speed" : "$gps.speed", 
                           "_id" : "$_id", 
                           "timestamp_local" : "$timestamp_local",
                           "gps_accuracy": "$gps.accuracy",
                           "latitude": "$gps.latitude",
                           "longitude": "$gps.longitude",
                           "elapsed_time": {"$multiply": ["$gps.number_of_samples", "$sample_period"]}
                           } },
            { "$project": { "_id": "$_id", 
                           "max_speed": { "$max": "$speed" }, 
                           "min_speed": { "$min": "$speed" }, 
                           "delta_lat": {"$subtract":[{ "$max": "$latitude" }, { "$min": "$latitude" }]},
                           "delta_long": {"$subtract":[{ "$max": "$longitude" }, { "$min": "$longitude" }]},
                           "max_lat": { "$max": "$latitude" },
                           "min_accuracy": {"$min": "$gps_accuracy"},
                           "timestamp_local" : "$timestamp_local",
                           "elapsed_time": "$elapsed_time"} },
            {"$match": {"max_speed": {"$gt": params["max_speed"]}, "min_speed": {"$lt": params["min_speed"]}, 
                        "min_accuracy": {"$lt": params["min_accuracy"]}, "delta_lat": {"$gt": params["delta_lat"]}} },
            {"$project": { "_id": "$_id", "timestamp_local" : "$timestamp_local", 
                          "elapsed_time": "$elapsed_time", "max_speed": "$max_speed", 
                          "delta_lat": "$delta_lat", "delta_long":"$delta_long", "max_lat": "$max_lat"} }                  
        ]

    agg=db.sensors.aggregate(agg_code)
    df=pd.DataFrame(agg)
    df["_id"]=df._id.apply(str)
    
    #filtering for the relevant times
    df["timestamp_local"]=df.apply(convert_str_to_datetime, axis=1)
    df=df[df.timestamp_local>=start_date]
    df=df[df.timestamp_local<=end_date]
    
    #sort by time
    df=df.sort_values(['timestamp_local'])
    df=df.reset_index(drop=True)
    
    # dropping sessions with too large distance (probably driving)
    dlat_km=np.array(df.delta_lat)*111.7
    dlong_km=np.array(df.delta_lat)*40075/360*np.cos(np.array(df.max_lat)/180*np.pi)
    dist_km=np.sqrt(dlat_km**2+dlong_km**2)
    dist_km2min=dist_km/np.array(df.elapsed_time)*60
    df=df.iloc[dist_km2min<params["max_dist_km2min"]]

    #finding where gap between sessions is high -> a new list
    t=np.array(df.timestamp_local)
    t2=np.array(df.timestamp_local.shift())
    dt=(t-t2)
    dt[0]=0
    dt=np.array([float(i) for i in dt])/10**9
    et=np.array(df.elapsed_time.shift())
    et[0]=0
    tot_dt=dt-et
    session_idx=np.concatenate([[0],np.where(tot_dt>params["gap_time"])[0],[len(t)]],axis=0)
    
    list_ids=list(df._id)
    list_of_lists=[]
    for i in range(len(session_idx)-1):
        list_i=list_ids[session_idx[i]:session_idx[i+1]]
        if len(list_i)>0:
            list_of_lists.append(list_i)
    return list_of_lists



In [5]:


import math

# {['latitude':1]},'gps_longitude':1 ,'gps_speed':1 

def read_VZ_from_mongo(_id):
    df_tmp=pd.DataFrame()
    dfjson=pd.DataFrame(db.sensors.find({"_id": ObjectId(_id)},{"_id": 1 ,'gps':1, 'timestamps':1}))
    #find number_of_samples
    vecs_dct={}
    singles_dct={}
    for column in dfjson.columns:
#        print(column)
        try:
            if dfjson[column][0]["number_of_samples"]>0:
                vecs_dct[column]=dfjson[column][0]["number_of_samples"]
        except:
            singles_dct[column]=dfjson[column][0]
    nos=min(vecs_dct.values()) #number_of_samples
    if max(vecs_dct.values())>min(vecs_dct.values()):
        print('NOTE: number_of_samples varies by {}'.format(max(vecs_dct.values())-min(vecs_dct.values())))
    for column in vecs_dct.keys():
        for key in dfjson[column][0].keys():
            if key!="number_of_samples" and len(dfjson[column][0][key])>=nos:
                col_name=column+'_'+key
                col_name=col_name.replace('bearing','azimuth')
                col_name=col_name.replace('testing_mode_value','testing_mode')
                df_tmp[col_name]=dfjson[column][0][key][:nos]
    for column in singles_dct.keys():
        df_tmp[column]=singles_dct[column]
    #correct and add columns
    
    #clean zeros in the lat/long reading
    df_tmp=df_tmp[df_tmp["gps_latitude"]<df_tmp["gps_latitude"].median()+1]
    df_tmp=df_tmp[df_tmp["gps_latitude"]>df_tmp["gps_latitude"].median()-1]
    def convert_str_to_datetime(row):
    #    print(row)
        try:
            return datetime.strptime(row['timestamps_value'], '%Y-%m-%dT%H:%M:%S.%f')
        except:
            t=row['timestamps_value']
            return datetime.strptime(t[:-3]+t[-2:], '%Y-%m-%dT%H:%M:%S.%f%z')

    df_tmp['timestamp']=df_tmp.apply(convert_str_to_datetime, axis=1)



    return df_tmp


In [6]:
#server.stop()

In [7]:
#pd.DataFrame(db.sensors.find({ 'user_id': "rZBbw12NtJYWsDqgomNJSg9z7ii1" })) 


In [8]:

def get_df_for_ids(ids):
        
    print(len(ids))
    # list_ids=list(df_walk._id)
    df_vz= pd.DataFrame()
    for _id in ids:
        try:
            df_tmp=read_VZ_from_mongo(_id)
            df_vz=pd.concat([df_vz,df_tmp], axis=0)
        except:
            print('problem with id {}'.format(_id))
        
#    df_vz['timestamp']=df_vz.apply(convert_str_to_datetime, axis=1)
    df_vz=df_vz.sort_values(['timestamp'])
    return df_vz.reset_index(drop=True)

In [11]:

def point_edge_distance(row):
    point= ge.Point(row['gps_longitude'],row['gps_latitude'])
    edge=row['nearest_edge']

    return point.distance(edge[0])*10000

def getExtrapoledLine(p1,p2):
    'Creates a line extrapoled in p1->p2 direction'
    EXTRAPOL_RATIO = 2
    a = p1
    a = (p1[0]-EXTRAPOL_RATIO*(p2[0]-p1[0]), p1[1]-EXTRAPOL_RATIO*(p2[1]-p1[1]) )
    b = (p1[0]+EXTRAPOL_RATIO*(p2[0]-p1[0]), p1[1]+EXTRAPOL_RATIO*(p2[1]-p1[1]) )
    return LineString([a,b])


def count_intersections(part_susp_cross):
    pos=0
    neg=0

    closest_point_index= part_susp_cross['dist_from_edge'].idxmin()
    for i in range(len(part_susp_cross)):
        try:
            while True:
                r1=random.randint(part_susp_cross.index[0],closest_point_index-1)
                if r1 in part_susp_cross.index:
                    break
            while True:
                r2=random.randint(closest_point_index+1,part_susp_cross.index[-1])
                if r2 in part_susp_cross.index:
                    break
            print(r1,r2)

            A=ge.Point( part_susp_cross.loc[r1]['gps_longitude'],  part_susp_cross.loc[r1]['gps_latitude'])
            B=ge.Point( part_susp_cross.loc[r2]['gps_longitude'],  part_susp_cross.loc[r2]['gps_latitude'])


            line1 = LineString([A, B])
            line2 = part_susp_cross.loc[closest_point_index]['nearest_edge'][0]
    #        print(line2.length*100000)

            if line2.length*100000 < 6:
                print('EXTENDED')
                line2=getExtrapoledLine(line2.coords[0],line2.coords[-1])
            int_ptr = line1.intersection(line2)
            if not int_ptr:
                neg+=1
            else:
                pos+=1
        except:
            return (closest_point_index,0)

    return (closest_point_index, pos/(pos+neg))

def nearest_edges(row, graph_edges):
    point= ge.Point(row['gps_longitude'],row['gps_latitude'])
    
    edges_with_distances = [
        (
            graph_edge,
            point.distance(graph_edge[0])
        )
        for graph_edge in graph_edges
    ]

    edges_with_distances = sorted(edges_with_distances, key=lambda x: x[1])
    return [e[0] for e in edges_with_distances][0:12]

def nearest_edge(row, map_indices_edges ):
    point= ge.Point(row['gps_longitude'],row['gps_latitude'])
    if row.name in map_indices_edges.index:
        return map_indices_edges.loc[row.name]['nearest_edges'][0]
    edges_to_check= map_indices_edges[map_indices_edges.index > row.name].iloc[0]['nearest_edges'] + map_indices_edges[map_indices_edges.index < row.name].iloc[-1]['nearest_edges'] 

    edges_with_distances = [
        (
            graph_edge,
            point.distance(graph_edge[0])
        )
        for graph_edge in edges_to_check
    ]

    edges_with_distances = sorted(edges_with_distances, key=lambda x: x[1])
    return [e[0] for e in edges_with_distances][0]


def crossings_info_from_df(df_AS):
    
    middle_loc=(df_AS['gps_latitude'].median(axis=0), df_AS['gps_longitude'].median(axis=0))
    max_dist=max(geopy.distance.distance((df_AS['gps_latitude'].max(axis=0), middle_loc[1]), middle_loc).m,
                geopy.distance.distance((df_AS['gps_latitude'].min(axis=0), middle_loc[1]), middle_loc).m,
                geopy.distance.distance((middle_loc[0], df_AS['gps_longitude'].max(axis=0)), middle_loc).m,
                geopy.distance.distance((middle_loc[0], df_AS['gps_longitude'].min(axis=0)), middle_loc).m)
    
    G = ox.graph_from_point(middle_loc, distance= max_dist+10, network_type='drive',simplify=False)
    
    print('map is ready')
    
    unique_locs_df= df_AS[['gps_longitude','gps_latitude']].drop_duplicates(['gps_longitude','gps_latitude']).reset_index()
    
    walk_pois=[ge.Point(row[1]['gps_longitude'],row[1]['gps_latitude']) for row in unique_locs_df.iterrows()]

    print('computing nearest edge...')
    
    walk_pois_df =df_AS[['gps_longitude','gps_latitude']].drop_duplicates(['gps_longitude','gps_latitude'])
    
    walk_pois_indices=list(walk_pois_df.index)

    ret_ce=[walk_pois_indices[0]]

    for i in walk_pois_indices:
        if i-100>ret_ce[-1]:
            ret_ce+=[i]
    ret_ce+=[walk_pois_indices[-1]]

    gdf = ox.graph_to_gdfs(G, nodes=False, fill_edge_geometry=True)
    graph_edges_ = gdf[["geometry", "u", "v"]].values.tolist()
    
    walk_pois_df.loc[ret_ce,'nearest_edges']=walk_pois_df.apply(nearest_edges, graph_edges=graph_edges_, axis=1)

    map_indices_edges = walk_pois_df[['nearest_edges']]

    map_indices_edges=map_indices_edges[pd.notnull(map_indices_edges['nearest_edges'])]

    del walk_pois_df['nearest_edges']

    walk_pois_df['nearest_edge'] = walk_pois_df.apply(nearest_edge,map_indices_edges=map_indices_edges, axis=1)
    
    walk_pois_df['dist_from_edge']=walk_pois_df.apply(point_edge_distance,axis=1)
    
    print('nearest edge is ready')
    
    unique_locs_df['last_index']=list(unique_locs_df['index'])[1:]+[len(df_AS)]

    
    unique_locs_df['med_index']=unique_locs_df['index']/2+unique_locs_df['last_index']/2
    

    print('graph edges is ready')

    walk_pois_df['dist_from_edge']=walk_pois_df.apply(point_edge_distance , axis=1)
    walk_pois_df=walk_pois_df.reset_index(drop=True)
    
    close_to_street_df= walk_pois_df[walk_pois_df['dist_from_edge'] < 0.3]
    close_to_street_df['prev_diff']=list(close_to_street_df.index)[1:]+[close_to_street_df.index.max()+1]
    
    susp_parts=close_to_street_df[close_to_street_df.index < close_to_street_df.prev_diff-2]
    


    print('computing intersections...')


    begin= 0
    end=close_to_street_df.index.max()
    if len(susp_parts)>0:
        end= susp_parts.index[0]


    closest_points_indices=[]
    numcrossing=0
    for i in range(len(susp_parts)+1):
        print(begin, ' to ', end)
        part_susp_cross=close_to_street_df[(close_to_street_df.index > begin) & (close_to_street_df.index <= end) ]
        intrate = count_intersections(part_susp_cross)
        if intrate[1]>0.25:
            closest_points_indices+=[intrate[0] ]
        print('--------------', intrate, '----------------')
        numcrossing+=intrate[1]
        begin=end
        if i<len(susp_parts)-1:
            end=susp_parts.index[i+1]
        else:
            end=close_to_street_df.index[-1]

    crossing_slices=list(unique_locs_df.loc[closest_points_indices]['med_index'].astype(int))
    res=[]
    for c in crossing_slices:
        res+=[(df_AS.loc[c-20:c]['gps_speed'].min(),df_AS.loc[c:c+40]['gps_speed'].mean())]
    return res

def cross_statistics(user_id, start_date, end_date,device_type):
    
    print('retreiving data from server')
    
    ids_sessions=get_id_list_from_user(user_id,  start_date, end_date, device_type)
    res=[]
    for ids in ids_sessions:
        print('get_df_for_ids')
        df_AS=get_df_for_ids(ids)
    
        print('dataframe is ready')
    
        res+=crossings_info_from_df(df_AS)
        print(len(res))
        return res


In [12]:
user_id='ms0Eg3WTooMYshw8vDA6tdTmJiM2'
start_date='2019-07-22 23:08:46+0300'
end_date='2019-07-22 23:30:40+0300'
device_type='iPhone 7'

res=cross_statistics(user_id, start_date, end_date, device_type)

retreiving data from server
get_df_for_ids
19
dataframe is ready
map is ready
computing nearest edge...
nearest edge is ready
graph edges is ready
computing intersections...
0  to  39
36 39
36 39
36 39
36 38
-------------- (37, 1.0) ----------------
39  to  80
-------------- (80, 0) ----------------
80  to  113
110 113
110 113
110 112
110 113
-------------- (111, 1.0) ----------------
113  to  152
149 152
148 152
149 152
150 152
149 152
-------------- (151, 1.0) ----------------
152  to  193
184 193
181 193
185 193
190 193
186 193
186 193
188 193
186 193
191 193
187 193
181 193
184 193
191 193
184 193
188 193
-------------- (192, 1.0) ----------------
193  to  261
237 256
238 250
237 252
237 249
237 242
238 245
237 243
238 248
238 260
237 251
237 258
238 254
237 259
238 247
237 253
237 255
237 255
237 243
237 244
237 248
238 247
237 243
238 246
237 241
237 257
-------------- (239, 1.0) ----------------
261  to  290
-------------- (289, 0) ----------------
290  to  295
-------------- (2

In [13]:
res

[(1.6519423723220825, 1.797991502575758),
 (1.1237138509750366, 1.4968441666626349),
 (1.4367644786834717, 1.4273992049984816),
 (1.3524123430252075, 1.6293143324735688),
 (0.9893452525138855, 1.5033913065747517),
 (0.859021008014679, 1.260362799574689),
 (1.4543136358261108, 1.4827891820814552),
 (1.5091376304626465, 1.3229935198295406),
 (1.1271804571151733, 1.2582713656309175),
 (0.2831747531890869, 1.5486501600684188),
 (1.6618789434432983, 1.4135305285453796),
 (0.8549215197563171, 1.4319524183505918),
 (1.464809536933899, 1.772305165849081),
 (1.237949252128601, 1.4454254813310576),
 (1.3882217407226562, 1.4474161776100718),
 (1.0647343397140503, 1.4604719121281693),
 (1.3752752542495728, 1.3319646498052085),
 (1.5988889932632446, 1.331000598465524),
 (1.1534191370010376, 1.2147941353117548),
 (1.2840059995651245, 1.401938333743956),
 (1.0646830797195435, 1.4241456956398197),
 (1.3157864809036255, 1.2356384382015322),
 (0.566231906414032, 1.2507894780577682),
 (1.1301981210708618